In [ ]:
# !pip3 install langchain_community

In [ ]:
from langchain_community.llms import Ollama

### Read transaction data

In [ ]:
# Read the transactions from csv
import pandas as pd
df = pd.read_csv("2024-05-22_transaction_download.csv")
df.head()

In [ ]:
#Get unique transactions in the Name / Description column
unique_transactions = df['Description'].unique()
len(unique_transactions)

In [ ]:
response = llm.invoke("Can you add an appropriate category to the following expenses. For example: SQ *RESENDIZ FARMS  - Groceries, CSC ServiceWorks, Inc. - Household Expenses, etc.. Categories should be less than 4 words. " + ",".join(unique_transactions[1:50]))
response = response.split("\n")

In [ ]:
# response

### Categorise bank transactions with Llama3

In [ ]:
# Get index list
#https://stackoverflow.com/questions/47518609/for-loop-range-and-interval-how-to-include-last-step
def hop(start,stop,step):
  for i in range(start,stop,step):
    yield i
  yield stop

index_list = list(hop(0, len(unique_transactions), 30))
index_list

In [ ]:
def categorize_transactions(transaction_names, llm):
    response = llm.invoke("Can you add an appropriate category to the following expenses. For example: SQ *RESENDIZ FARMS  - Groceries, CSC ServiceWorks, Inc. - Household Expenses, Aegean - Travel etc.. Categories should be less than 4 words, description and category seperated by a dash. Some places might have japanese names if category can't be found they can be specified as Travel, especially if it's during March 26 and April 9." + transaction_names)
    response = response.split('\n')

    print(response)

    # Put in dataframe
    categories_df = pd.DataFrame({'Transaction vs category': response})

    # New code to handle strings without ' - '
    def split_and_fill(s):
        parts = s.split(' - ', maxsplit=1)
        if len(parts) == 1:
            parts.append('NaT')  # Default category
        return pd.Series(parts)

    categories_df[['Transaction', 'Category']] = categories_df['Transaction vs category'].apply(split_and_fill)

    return categories_df

In [ ]:
# Test out the function
categorize_transactions('ISS Catering Services De Meern, Taxi Utrecht, Etos AMSTERDAM NLD, Bistro Bar Amsterdam',
                        llm)

In [ ]:
# Intialise the categories_df_all dataframe
categories_df_all = pd.DataFrame()

# Loop through the index_list
for i in range(0, len(index_list)-1):
    transaction_names = unique_transactions[index_list[i]:index_list[i+1]]
    transaction_names = ','.join(transaction_names)

    categories_df = categorize_transactions(transaction_names, llm)
    categories_df_all = pd.concat([categories_df_all, categories_df], ignore_index=True)

In [ ]:
categories_df_all


In [ ]:
# categories_df_all.to_csv("categories_df_all.csv", index=False)

In [ ]:
# Get unique categories in categories_df_all
unique_categories = categories_df_all["Category"].unique()
unique_categories

In [ ]:
# Drop NA values
categories_df_all = categories_df_all.dropna()

# If category contains "Food", then categorise as "Food and Drinks"
categories_df_all.loc[categories_df_all['Category'].str.contains("Food|Bar|Pub"), 'Category'] = "Food and Drinks"
# If category contains "Clothing", then categorise as "Clothing"
categories_df_all.loc[categories_df_all['Category'].str.contains("Clothing"), 'Category'] = "Clothing"
# If category contains "Services", then categorise as "Services"
categories_df_all.loc[categories_df_all['Category'].str.contains("Services"), 'Category'] = "Services"
# If category contains "Health" or "Wellness", then categorise as "Health and Wellness"
categories_df_all.loc[categories_df_all['Category'].str.contains("Health|Wellness"), 'Category'] = "Health and Wellness"
# If category contains "Sport", then categorise as "Sport
#  and Fitness"
categories_df_all.loc[categories_df_all['Category'].str.contains("Sport"), 'Category'] = "Sport and Fitness"
# If category contains "Travel", then categorise as "Travel"
categories_df_all.loc[categories_df_all['Category'].str.contains("Travel"), 'Category'] = "Travel"
# If category contains "Transport", then categorise as "Transport"
categories_df_all.loc[categories_df_all['Category'].str.contains("Transport|Transportation|Gasoline"), 'Category'] = "Transport"

In [ ]:
# Remove the numbering eg "1. " from Transaction column
categories_df_all['Transaction'] = categories_df_all['Transaction'].str.replace(r'\d+\.\s+', '')
categories_df_all.to_csv("categories_df_all.csv", index=False)

In [ ]:
# Merge the categories_df_all with the transactions_2022_2023.csv dataframe (df)
# df = pd.read_csv("2024-05-22_transaction_download.csv")
# df.loc[df['Transaction Description'].str.contains("Spotify"), 'Name / Description'] = "Spotify Ab By Adyen"
# df = pd.merge(df, categories_df_all, left_on='Transaction Description', right_on='Transaction Description', how='left')
# df